# A/B Test Analysis

This notebook replicates a simple A/B test workflow using synthetic data.

In [ ]:

import pandas as pd
from src.data_generator import generate_ab_data
from src.ab_test import group_summary, propor_z_test
from src.visualization import plot_cr_with_ci

# Load synthetic data
df = pd.read_csv('../data/ab_test_data.csv')
df.head()


## Per-group conversion and 95% CI

In [ ]:

summary = group_summary(df, group_col='group', outcome_col='converted')
summary


In [ ]:

# Plot conversion rate with 95% CI
plot_cr_with_ci(summary, group_col='group', cr_col='cr', se_col='se')


## Two-proportion z-test

In [ ]:

result = propor_z_test(df, group_col='group', outcome_col='converted', control='A', treatment='B')
result


### Plain-English Recommendation

In [ ]:

p1 = result['p_control']*100
p2 = result['p_treatment']*100
uplift = result['uplift']*100
lo = result['uplift_ci_low']*100
hi = result['uplift_ci_high']*100
sig = result['p_value'] < 0.05

print(f"CR(A) = {p1:.2f}%, CR(B) = {p2:.2f}%")
print(f"Uplift = {uplift:.2f}% (95% CI: {lo:.2f}% to {hi:.2f}%)")
print(f"p-value = {result['p_value']:.4f} -> {'SIGNIFICANT' if sig else 'NOT significant'}")

if sig and uplift > 0:
    print("Recommendation: Roll out Variant B.")
elif sig and uplift < 0:
    print("Recommendation: Keep A (B underperforms).")
else:
    print("Recommendation: Inconclusive – extend the test or collect more data.")
